Welcome to my JAX tutorial series. This is the fourth part of this series. As you notice, wherever you look at, you become encountered by something about JAX in machine learning domain, and naturally you become curious about what is really that JAX thing. I am here to explain and clarify your curiosity in the most detailed way. You can find the list of my tutorials below:


**JAX Tutorials:**

* [1. Introduction to JAX](https://www.kaggle.com/code/goktugguvercin/introduction-to-jax)
* [2. Gradients and Jacobians in JAX](https://www.kaggle.com/code/goktugguvercin/gradients-and-jacobians-in-jax)
* [3. Automatic Differentiation in JAX](https://www.kaggle.com/code/goktugguvercin/automatic-differentiation-in-jax)
* [4. Just-In-Time Compilation in JAX](https://www.kaggle.com/code/goktugguvercin/just-in-time-compilation-in-jax)


<div style="width:100%;text-align: center;"> 
<img align=middle src="https://raw.githubusercontent.com/google/jax/main/images/jax_logo_250px.png" width="250" height="250">
</div>

In this tutorial, we will mainly focus on just-in-time (JIT) compilation system used by JAX. To understand JIT mechanics in JAX properly, I will try to explain some level details and concepts like *tracing* and *jaxpr* in JIT. Then, I will describe in which conditions just-in-time compilation is applicable or not. 

In [1]:
import jax
import jax.numpy as jnp

# Compilation System in XLA:

JAX takes advantage of XLA to compile and run its numpy code on the accelerators like GPU and TPU. To get better understanding about Just-In-Time (JIT) compilation property of JAX and to figure out how it helps to accelerate the execution of functions, we need to briefly investigate XLA compilation system. 

XLA, standing for Accelerated Linear Algebra, is a domain specific compiler introduced by Google to both leverage computational efficiency and reduce memory consumption for machine learning models. It allows the operations to be emitted and executed on multiple backends like CPU, GPU, and TPU. However, what really qualifies XLA to have computational and memory-based improvements is the functional language system that it uses to represent linear algebra computations, which is called High-Level Operations-Intermediate Representations (HLO-IR). During the compilation phase of python functions, XLA, first of all, creates HLO-IR based expressions to symbolize those functions. Then, XLA rewrites these expressions to modify and optimize execution of corresponding symbolic functions. At this point, XLA has a series of optimization techniques defined on its functional language system, but one of the most powerful techniques is operation-fusion, which is to combine multiple computations. 

First of all, operation fusion does not write intermediate results of the computation to the memory; instead prefers to store them in registers. In that way, memory requirements are highly alleviated. In addition to this, the fusion of multiple operations makes their compilation and execution together in just one CUDA kernel. Otherwise, the computations would be individually evaluated at different kernels, each of which is dedicated to one operation. This is actually quite inefficient; fusion-based optimization reduces the number of kernels to be launched. What makes HLO-IR so powerful and good at optimization is that it is **strongly-typed** with exact data type and exact shape. In other words, if the dimensions and data type of your tensors are not static; instead they need to be inferred, then XLA cannot compile and naturally fuse them. 

At this point, you probably got the idea lying behind XLA, but you may start to ask yourself what kind of relationship exists between XLA and JIT compilation. In fact, XLA is a just-in-time compiler and without JIT property, all that operation optimization procedure becomes limited. That's why we can just-in-time compile JAX functions and thereby accelerate them.

# Just-In-Time Compilation:

Static compilation, also called as "ahead-of-time compilation", aims to convert a program written in high level programming language into quite primitive form in object code or assembly system before that program is executed. The main reason why it attempts to finish compilation at build-time rather than just doing this together with the execution at the same time is to reduce total amount of workload at runtime. On the other hand, interpreters prefer to directly execute the instructions and operations in target program; the translation of the program to machine language is not required in working structure of interpreters. Just-in-time compilation actually tries to combine the advantages of these two techniques: First of all, it interpretes the program to be easily executed, but then, when the execution of the program starts, it tries to spot frequently used segments of the program like iterations and compiles them to machine code in order to provide additional acceleration. At this point, compilation is, in fact, carried out on the fly and in parallel with the execution; hence, you may be encountered by the phrase ***dynamic compilation*** in place of JIT. 

# The Role of JIT in JAX:

To bring the advantages of dynamic compilation and code optimization techniques of XLA engine together, JAX provides us with quite handy transformer [$jax.jit()$](https://jax.readthedocs.io/en/latest/_autosummary/jax.jit.html#jax.jit). It takes a function as input and returns its wrapped version to be ready for compilation and optimization. In other words, $jax.jit()$ does not compile the given function; instead it carries out some preparatory operations for it to be jit-compiled correctly. Compilation and optimization, in fact, are performed by JIT system during the first call to the function returned by $jax.jit()$. 

In the following code block, $tanh()$ activation function is defined and then it is jit-compiled. Since jit-compilation happens in the first call, total elapsed time is measured on the second call. During the compilation process, the tracers are at first materialized over abstract values of inputs, and then these tracers are used to define a jaxpr instance which represents function context. At this point, XLA compiles that jaxpr object into efficient code optimized for accelerators. Finally, optimized context of the function is cached for further calls to $jit\_tanh()$. Making time measurement and performance analysis over first call means the inclusion of total duration passed during tracing and compilation. To make fair and more correct comparison, compilation time should not be included in the benchmark. Let's look at how tanh activation and its jit version will behave for 1 million score values:

In [2]:
# tanh activation function
def tanh(scores):
    return (jnp.exp(scores) - jnp.exp(-scores)) / (jnp.exp(scores) + jnp.exp(-scores))

# score values, that will be activated by tanh, are sampled
key = jax.random.PRNGKey(137)
scores = jax.random.uniform(key, (1000000, ))

# wrapped version of tanh 
tanh_jit = jax.jit(tanh)

print("Total duration required for standard tanh: ")
%time tanh(scores).block_until_ready()

# First call
# Warm up phase: creation of jaxpr by tracers
# Compilation and optimization happen in this line
%time tanh_jit(scores).block_until_ready()

# Second call
print("\nTotal duration required for optimized tanh by XLA: ")
%time tanh_jit(scores).block_until_ready()

Total duration required for standard tanh: 
CPU times: user 182 ms, sys: 19.4 ms, total: 201 ms
Wall time: 186 ms
CPU times: user 44.2 ms, sys: 789 µs, total: 45 ms
Wall time: 43.6 ms

Total duration required for optimized tanh by XLA: 
CPU times: user 3.3 ms, sys: 0 ns, total: 3.3 ms
Wall time: 1.49 ms


DeviceArray([0.6470789 , 0.6376749 , 0.5518919 , ..., 0.6078519 ,
             0.41909885, 0.20145698], dtype=float32)

Total duration is quite changable depending on the type and also availability of backend that you use. When I executed the code above in my computer CPU, total elapsed time was $17.7$ miliseconds and $2.7$ miliseconds for standard and optimized tanh activators respectively. This means that XLA optimization provides us with approximately $8$ times faster execution. On the other hand, when I tested same code on Google Colab GPU device, I was encountered by $4$ times better results (657 microseconds and 1.9 miliseconds).

# Side-Effects, Tracing and Creation of Jaxpr Instance in JIT Mechanics:

As mentioned in previous part, tracing and creation of jaxpr eventually are essential assets required for jit compilation to be successfully conducted. To understand how they exactly work, we need to look at the fundamental mechanism lying behind function transformers, which actually relies on statically-typed expression oriented language introduced by JAX, which is $jaxpr$. Jaxpr works as an intermediary at this point. First of all a representative jaxpr object is created for the python function to be transformed. During the instantiation of jaxpr, each argument of that python function is wrapped by a $tracer$ object, and then each of those tracers takes a record of all operations in which its wrapping argument is included. Composition of those records eventually bears our jaxpr instance. What makes jaxpr so important is the fact that compilation process is carried out over it. At this point, XLA enters the picture; the computational expression in jaxpr is so basic and primitive, but XLA improves it and eventually make it much more efficient. In other words, the jaxpr is compiled over XLA to be optimized for GPU and TPU. Let's examine this entire process in more detail with a small Q&A session:

1. When do tracing and creation of jaxpr happen?
*Tracing and creation of jaxpr are performed in the first call to the function returned by $jax.jit()$*

2. Is the first call to the function $jit\_tanh()$ executed on its optimized, jit-compiled version or first execution is completely carried out over its primitive, unoptimized python form?

*Tracing and creation of jaxpr are performed with abstract values instantiated to represent real input arguments. When these two operations are completed, compilation over jaxpr by using XLA is initiated. In final step, real inputs, not their abstract representatives, are passed to compiled function for execution. In other words, execution of first call is realized on jit-compiled function.*

3. what if a function with side-effects tries to be just-in-time compiled?

*To be able use just-in-time compilation system in JAX correctly, the functions passed to $jax.jit()$ have to be deprived of side-effects. In other words, as mentioned in previous notebooks, JAX follows the principles of functional programming paradigm; hence, all compilation and transformation systems in JAX are designed to work only on pure functions. If an impure function is passed to $jax.jit()$, tracers do not record its side effects code blocks inside function context during the construction of jaxpr object; however, they need to execute them just once. Hence, the first call to jit-compiled impure function executes side effects, but since jaxpr is created in side-effect free form, the subsequent calls do not contain execution of any side-effect. To be more informed about pure function and in-place update in JAX, you can check [JAX - The Sharp Bits](https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#pure-functions). What will happen when a function with side-effects is passed to $jax.jit()$ is also demonstrated in the following code block:*

I learnt so many details about JIT mechanics explained in this QA session with [this discussion](https://github.com/google/jax/discussions/10430#discussioncomment-2625602), so I am grateful to Jake Vanderplas and his clarifications. 

In [3]:
# It counts how many multiplications are performed
counter = 0

# It multiplies two values.
# Then, it increments the counter by 1 to count the number of operations done up to now.
# There is no point to jit compile this function, since it does not have heavy workload.
# It is to show the behavior of side-effects.
def multiply(item1, item2):
    global counter
    result = item1 * item2
    counter = counter + 1
    return result

jit_multiply = jax.jit(multiply)

print("First call to jit_multipy(): ", jit_multiply(5, 4))  # counter is incremented by 1
print("Second call to jit_multiply(): ", jit_multiply(2, 13))  # counter is not incremented by 1
print("Counter: ", counter)

First call to jit_multipy():  20
Second call to jit_multiply():  26
Counter:  1


# Applicability Conditions of Just-In-Time Compilation in JAX:

 Just-In-Time (JIT) compilation in JAX comes up with some applicability conditions and constraints required to be satisfied so that JIT mechanics can work well without any problem and the execution of jit-compiled functions can become much more efficient. These conditions and constraints actually decide on whether the functions are compatible with JIT or not. Let's we look at and investigate each of them one by one: 

***Static Shapes and Data-Types:***

The first and maybe the most important one of these conditions is the fact that arrays need to have static shapes and data-types. In other words, JAX requires the shape and data-type of arrays to be known at compile time. The main reason for this is to increase the reusability of jit-compiled and cached functions. 

During JIT-compilation, as mentioned before, abstract values are instantiated to represent real input arguments of target function, and the tracers for these values are defined to keep track of all operations performed on them. The important thing at this point is the level of abstraction to represent real arguments. According to JAX-official tutorials, "by default, jit traces your code on the ShapedArray abstraction level, where each abstract value represents the set of all array values with a fixed shape and dtype." In other words, abstract values need to know the shape and data type of real input arguments, but they do not contain concrete values of those inputs. You can think like abstract values are placeholders for the arguments that will be passed the jit-compiled function.  

For example, we define a function for matrix-multiplication, and we want to jit-compile this function. First call, as mentioned above, is the time when compilation is conducted. If we pass two floating point matrices like $(100, 50)$ and $(50, 150)$ during first call, these matrix shapes and floating-point data type will be saved to the cache. As a result, jit-compiled version of that function becomes available and reusable for many different matrices with the shape of $(100, 50)$ and $(50, 150)$. In other words, you create a template function with some assumptions about input arguments, and you can reuse that template in a very efficient way without additional recompliation costs for all input values that obey those assumptions. You can imagine this as a class-object relationship in object-oriented programming. 

At this point, you can ask what if we do not obey these rules, and pass the matrices with different shapes. In this case, the function is compiled again and the result of new compilation is cached with those shapes. To be able benefit from reusability and save on re-compilations, it is good to obey the rules dictated by abstraction level. However, there are different levels of abstraction (UnshapedArray - ShapedArray - ConcreteArray); we can control and decide which one will be applied during compilation. In that way, we can create more geneal and more specific function template in the cache. To get more details about this, you can look at [my discussion](https://github.com/google/jax/discussions/10518) with Jake Vanderplas and [source code](https://github.com/google/jax/blob/main/jax/_src/abstract_arrays.py) about abstract arrays in JAX.

Last interesting and also noticable point is whether the usage of such an abstraction level (ShapedArray) is really senseful and reasonable or not. I can answer this question with this example: When we define neural networks with bunch of layers, as you know, these layers have weight matrices, and kernel filters, with fixed shape and data type. We only update the content of these matrices and filters during training, we do not change their shape or we do not convert their data-type from floating point to integer. The functions defined for training and evaluation steps of neural networks in Flax are; therefore, jit-compiled. To sum up, this abstraction level for such stable operations provides great efficiency.

In [4]:
key = jax.random.PRNGKey(13)
key1, key2 = jax.random.split(key)

def matrix_product(matrix1, matrix2):
    print(matrix1)
    result = matrix1 @ matrix2
    return result

jit_matrix_product = jax.jit(matrix_product)

# first series of input arguments
matrix1 = jax.random.uniform(key1, (100, 120))
matrix2 = jax.random.uniform(key1, (120, 100))

# second series of input arguments
matrix3 = jax.random.uniform(key1, (100, 120))
matrix4 = jax.random.uniform(key1, (120, 100))

# third series of input arguments
matrix5 = jax.random.uniform(key2, (200, 200))
matrix6 = jax.random.uniform(key2, (200, 300))

# * Total duration of first execution is long due to compilation time
# * Total duration of second execution is short since we obey shape rules and reuse compiled function in cache
# * Total duration of third execution is long, since we do not obey shape rules and function is compiled again
%time jit_matrix_product(matrix1, matrix2).block_until_ready()
%time jit_matrix_product(matrix3, matrix4).block_until_ready()
%time jit_matrix_product(matrix5, matrix6).block_until_ready()

Traced<ShapedArray(float32[100,120])>with<DynamicJaxprTrace(level=0/1)>
CPU times: user 47.6 ms, sys: 5.97 ms, total: 53.6 ms
Wall time: 69.1 ms
CPU times: user 11 µs, sys: 1.01 ms, total: 1.02 ms
Wall time: 730 µs
Traced<ShapedArray(float32[200,200])>with<DynamicJaxprTrace(level=0/1)>
CPU times: user 22.9 ms, sys: 1.97 ms, total: 24.8 ms
Wall time: 26.2 ms


DeviceArray([[44.07261 , 48.75547 , 47.686485, ..., 44.920574, 44.23801 ,
              46.823452],
             [46.590187, 49.964134, 50.3746  , ..., 51.567867, 46.55512 ,
              51.885838],
             [45.30676 , 48.752483, 48.470543, ..., 47.045856, 47.377876,
              48.55696 ],
             ...,
             [46.079403, 51.5776  , 47.79821 , ..., 50.519505, 47.708683,
              49.835613],
             [47.860855, 53.47438 , 51.01819 , ..., 51.48336 , 51.073547,
              51.8239  ],
             [45.920753, 49.756104, 46.528652, ..., 51.699642, 48.050743,
              49.653404]], dtype=float32)

***Function Context independent from Concrete Values of Input Arguments***

In some functions, we need to create control flows in which the behavior of the function changes depending on the values of input arguments passed to that function, and to achieve this, we define conditional expressions over those input values. In fact, such functions are not allowed to be jit-complied, since as highlighted just above, abstract values of tracers have only concrete shape and concrete data type, but no concrete value. When JIT-compiler faces a line like $if$ $(x > 5)$, the expression $x > 5$ is attempted to be evaluted on abstract value of input x; however, since it does not contain a concrete value, the evaluation fails, and tracing is stopped there. 

Likewise, if we attempt to jit-compile a function which creates a numpy array with a shape determined by the input passed to that function, we get encountered by same problem. During jit-compilation, shape input would be represented by an abstract value without no concrete context; hence, it is not possible to determine the shape of that array in advance. This automatically stops the tracing and jit-compilation directly fails. You can check how JIT behaves for these cases with the following codes:

In [5]:
key = jax.random.PRNGKey(seed=13)
array = jnp.array([i for i in range(100)])
rand_values = jax.random.choice(key, array, shape=(50,))

# * It is a piece-wise mathematical function.
# * Its behavior changes depending on the input.
# * It defines a conditional expression on its input x.
def piecewise(x):
    if x < 3:
        return 3. * x ** 2
    else:
        return -4 * x

# * It is an iteration function.
# * It iterates over a series of random values, and pick first n values.
# * How many values it will pick depends on its input n.
def iterate_until(n):
    my_list = []
    for i in range(n):
        my_list.append(rand_values[i])
    
    return my_list

# * It is a function used for creating a matrix of particular shape.
# * The shape of matrix that it will create depends on its input argument.
def create_matrix(shape):
    return jnp.ones(shape)


# * At this point, we don't witness any kind of error, since jit-compilation
# is performed on first call. jax.jit() only performs preparatory operations for jit-compilation. 
piecewise_jit = jax.jit(piecewise)
iterate_until_jit = jax.jit(iterate_until)
create_matrix_jit = jax.jit(create_matrix)

# * First call to any of them creates an error.
# * You can look at each of them one by one.
# * Main problem, as mentioned above, is that context of all these functions 
# depends on concrete values of input arguments. However, during JIT, their abstract copies;
# are used, and they don't contain any concrete value. Hence, tracing stopped there. 
try:
    print(piecewise_jit(5))
except Exception as e:
  print("Exception {}".format(e))

try:
    print(iterate_until_jit(20))
except Exception as e:
  print("Exception {}".format(e))

try:
    print(create_matrix_jit(30,))
except Exception as e:
  print("Exception {}".format(e))

Exception Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
The problem arose with the `bool` function. 
While tracing the function piecewise at /tmp/ipykernel_19/3636844857.py:8 for jit, this concrete value was not available in Python because it depends on the value of the argument 'x'.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError
Exception The __index__() method was called on the JAX Tracer object Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerIntegerConversionError
Exception Shapes must be 1D sequences of concrete values of integer type, got (Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=0/1)>,).
If using `jit`, try using `static_argnums` or applying `jit` to smaller subfunctions.



At this point, we conclude that function context needs to be completely independent from concrete values of input arguments. Besides, having static shape and data type for inputs provides additional efficiency by getting rid of additional re-compilations. In this case, you may ask how we can get around these constraints:

* Fixed shape and fixed data type constraints do not prevent jit-compilation process. Even if we violate them, just the function is jit-compiled again, but in this case, we cannot take the savings on recompliations.

* Input-dependent function context; however, is a situation that we really need to care about. Either we do not jit-compile these kinds of functions, or we can **order JIT mechanics not to trace input values that the behavior of the function depends on**. The transformer $jax.jit()$ has two differents parameter called "static_argnums" and "static_argnames" dedicated to this. These parameters allow us to determine which input arguments will be static and which ones will be traced. In the function template to be cached, static arguments have a concrete fixed value, while traced ones continue to be represented by abstract values. In other words, generalizability of function template is reduced for static arguments. However, the function needs to be recompiled for every new value of static arguments. As mentioned in official JIT documentation of JAX, "It is only a good strategy if the function is guaranteed to get limited different values for static inputs."

In [6]:
# The parameter "static_argnums" takes a tuple of indices to specify which input arguments would be static
# The parameter "static_argnames" takes a list of names to specify which input arguments would be static
piecewise_jit = jax.jit(piecewise, static_argnums=(0))
iterate_until_jit = jax.jit(iterate_until, static_argnames=["n"])
create_matrix_jit = jax.jit(create_matrix, static_argnums=(0))

# All these three functions are properly jit-compiled during the first call. 
# ConcretizationError is not monitored.
print(piecewise_jit(5))
print()

print(iterate_until_jit(7))
print()

print(create_matrix_jit(30,))
print()

-20

[DeviceArray(15, dtype=int32), DeviceArray(85, dtype=int32), DeviceArray(30, dtype=int32), DeviceArray(93, dtype=int32), DeviceArray(77, dtype=int32), DeviceArray(42, dtype=int32), DeviceArray(4, dtype=int32)]

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1.]



When we attempt to jit-compile the functions $piecewise()$, $iterate\_until()$ and $create\_matrix()$, we are faced with [*ConcretizationError*](https://jax.readthedocs.io/en/latest/errors.html). The main reason for this is that   JAX tracer is used in a scope where a concrete value is required. The parameters *argnums* and *argnames* eliminate the abstraction on specified input arguments, which makes them static in cache. In that way, those arguments could have a concrete value instead of being represented by abstract instances in tracing phase of jit-compilation, and all control flows dependent on those arguments become executable.

# References

**XLA Engine and Just-In-Time Compilation:**
* https://www.tensorflow.org/xla
* https://www.youtube.com/watch?v=cPAD9vLKE0c&t=223s
* https://en.wikipedia.org/wiki/Just-in-time_compilation
* https://jax.readthedocs.io/en/latest/notebooks/quickstart.html

**JIT, Tracing, Jaxpr Instance, Side Effects in JAX:**
* https://jax.readthedocs.io/en/latest/_autosummary/jax.jit.html#jax.jit
* https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html#how-jax-transforms-work
* https://jax.readthedocs.io/en/latest/jaxpr.html
* https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#pure-functions
* https://jax.readthedocs.io/en/latest/jax-101/07-state.html

**Applicability Conditions of Just-In-Time Compilation in JAX:**
* https://jax.readthedocs.io/en/latest/jax-101/02-jitting.html#why-can-t-we-just-jit-everything
* https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html#jit-mechanics-tracing-and-static-variables
* https://jax.readthedocs.io/en/latest/notebooks/Common_Gotchas_in_JAX.html#control-flow
* https://jax.readthedocs.io/en/latest/notebooks/How_JAX_primitives_work.html#jit
* https://jax.readthedocs.io/en/latest/errors.html